In [ ]:
# This file contains all the code that was used for testing purposes but was either not necessary or replaced
#for the final implementation and for the experiment

In [ ]:
#Get sample sentiment tweets to create classifier

#positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
#negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_tweet_tokens = twitter_samples.strings('positive_tweets.json')
negative_tweet_tokens = twitter_samples.strings('negative_tweets.json')


positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(clean_data(tokens))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(clean_data(tokens))

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [ ]:
#Train sentiment model
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

In [ ]:
#Suffle dataset randomly and train the model
random.shuffle(dataset)
train_data = dataset[:7000]
test_data = dataset[7000:]

In [ ]:
#Test sentiment classifier model on sample data
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(20))


In [ ]:
#Test the accuracy of the classifier on my data
correct = 0
neutral = 0
total = df['text'].size
for i in range(df['text'].size):
    model_sentiment = classifier.classify(dict([token, True] for token in df.loc[i, 'text']))
    if (df.loc[i, 'sentiment'] == 0):
        neutral = neutral + 1
    elif (model_sentiment == 'Positive' and df.loc[i, 'sentiment'] > 0):
        correct = correct + 1
    elif (model_sentiment == 'Negative' and df.loc[i, 'sentiment'] < 0):
        correct = correct + 1

print(correct)
print(neutral)
accuracy = 100 * (correct / (total - neutral))
print("accuracy is: " + str(accuracy) + "%")

In [ ]:
#Data cleaning functions 
stop_words = set(stopwords.words("English"))
#add stemmed stopwords to the stopwords set
stop_words.add("thi")
stop_words.add("whi")

def clean_data(raw_text):
    #tokenize
    filtered_sentence = []
    tknzr = TweetTokenizer()
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    words = tknzr.tokenize(raw_text)
    tagged = nltk.pos_tag(words)
    for w, tag in tagged:
        #remove noise
        w = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', w)
        w = re.sub("(@[A-Za-z0-9_]+)",'', w)
        w = re.sub('RT','',w)
        #POS
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        #Lemmatize and stem
        clean_word = ps.stem(lemmatizer.lemmatize(w, pos))
        #filter stop words, punctuation and blank space
        if clean_word.lower() not in stop_words and len(clean_word) > 0 and clean_word not in string.punctuation:
            filtered_sentence.append(clean_word.lower())
    return filtered_sentence

def clean_data2(raw_text):
    #tokenize
    filtered_sentence = []
    tknzr = TweetTokenizer()
    
    words = tknzr.tokenize(raw_text)
    for w in words:
        #remove noise
        w = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', w)
        w = re.sub("(@[A-Za-z0-9_]+)",'', w)
        w = re.sub('RT','',w)

        #filter stop words, punctuation and blank space
        if w.lower() not in stop_words and len(w) > 0 and w not in string.punctuation:
            filtered_sentence.append(w.lower())
    return filtered_sentence

In [ ]:
#clean the data for spaCy 

tic = time.perf_counter()
condition = aprilDF['date'] == '01-04-2020'
K = aprilDF[condition]['text']
x = 0


for row in K:
    K.at[x] = ' '.join(clean_data2(row))
    x = x + 1
    
toc = time.perf_counter()
print(toc -tic)

In [ ]:
#SpaCY token Matcher
tic = time.perf_counter()
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [{"LOWER": "travellers"}]
matcher.add("HelloWorld", [pattern])
condition = aprilDF['date'] == '01-04-2020'
totalMatches = 0
for rowText in aprilDF[condition]['text']:
    doc = nlp(rowText)
    matches = matcher(doc)
    totalMatches = totalMatches + len(matches)

toc = time.perf_counter()

print(toc - tic)

In [ ]:
#Testing if doing loading in the data as one big string is faster 
tic = time.perf_counter()
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [{"LOWER": "covid"}]
matcher.add("COVID", [pattern])



doc = nlp(J[:1000000])
matches = matcher(doc)

toc = time.perf_counter()

print(toc - tic)
print(len(matches))